In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7d98a9d623e44bb1af907ee9c82974c0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='DftPeV_h5Cmw417QAhQneXNdMpwtsU6phlo2DJdUWvmG',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_7d98a9d623e44bb1af907ee9c82974c0.get_object(Bucket='moviesuccessrate-donotdelete-pr-i6raeqovbhijin',Key='moviepredictor.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
dataset.drop(['color','num_critic_for_reviews','duration','director_facebook_likes','actor_3_facebook_likes','actor_1_facebook_likes'],inplace=True,axis=1)

In [3]:
dataset.drop(['cast_total_facebook_likes','actor_3_name','facenumber_in_poster','language','country','content_rating','actor_2_facebook_likes','aspect_ratio','movie_facebook_likes'],inplace=True,axis=1)

In [4]:
dataset.drop(['movie_imdb_link','title_year','plot_keywords'],axis=1,inplace=True)

In [5]:
dataset.drop(['num_user_for_reviews'],inplace=True,axis=1)

In [6]:
dataset=dataset.iloc[0:29,:]

In [7]:
dataset['gross'].fillna(dataset['gross'].mode(),inplace=True)

In [8]:
dataset['budget'].fillna(dataset['budget'].mean(),inplace=True)

In [9]:
dataset.isnull().any()

director_name      False
actor_2_name       False
gross              False
genres             False
actor_1_name       False
movie_title        False
num_voted_users    False
budget             False
imdb_score         False
dtype: bool

In [10]:
dataset.drop(['director_name','actor_2_name','genres','actor_1_name','movie_title'],axis=1,inplace=True)

In [11]:
dataset.isnull().any()


gross              False
num_voted_users    False
budget             False
imdb_score         False
dtype: bool

In [12]:
x=dataset.iloc[:,:-1]

In [13]:
y=dataset.iloc[:,-1:]

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [15]:
from sklearn.linear_model import LinearRegression
mr=LinearRegression()

In [16]:
mr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [17]:
y_predict=mr.predict(x_test)

In [18]:
y_predict

array([[6.91667921],
       [7.08515735],
       [7.05784144],
       [7.3518864 ],
       [6.71519424],
       [6.78727556]])

In [19]:
y_test

,imdb_score
2,6.8
20,7.5
23,7.9
13,7.3
11,6.1
25,7.2


In [20]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [21]:
wml_credentials={
  "apikey": "ZvZQuK2KTYK0dDrwfe5QwfdQk3AZd7vzwNmOKDsV00_y",
  "iam_apikey_description": "Auto-generated for key 3a206fa1-a3e2-4ece-b3bf-e5b8765cfb49",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/78fd8f135f9f475fa08d92d89a5b755f::serviceid:ServiceId-3a4e6b82-5ee8-4fb7-8b2f-0f5dfb39eb06",
  "instance_id": "37f4c38a-ec1d-4630-8ead-7be4e2481b71",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [22]:
client=WatsonMachineLearningAPIClient(wml_credentials)

In [23]:
metadata={
    client.repository.ModelMetaNames.DESCRIPTION:'movie rating',
    client.repository.ModelMetaNames.AUTHOR_NAME:'KARTHIK',
    client.repository.ModelMetaNames.NAME:'MULTILINEAR',
    client.repository.ModelMetaNames.FRAMEWORK_NAME:"scikit_learn",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION:"0.19",
}

In [28]:
model_details=client.repository.store_model(mr,meta_props=metadata)

model_details=client.repository.store_model(mr,met_props=metadata)

In [29]:
published_model_uid=client.repository.get_model_uid(model_details)
published_model_uid

'b484e856-63d1-46a7-97f8-2314dfe16f0d'

In [30]:
created_deployment=client.deployments.create(published_model_uid,name='multilinearregression')



#######################################################################################

Synchronous deployment creation for uid: 'b484e856-63d1-46a7-97f8-2314dfe16f0d' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6452ab90-5634-49ea-ac6a-f74d2c8064cc'
------------------------------------------------------------------------------------------------




In [31]:
scoring_endpoint=client.deployments.get_scoring_url(created_deployment)
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/37f4c38a-ec1d-4630-8ead-7be4e2481b71/deployments/6452ab90-5634-49ea-ac6a-f74d2c8064cc/online'

In [32]:
client.deployments.list()

------------------------------------  ---------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                   TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
6452ab90-5634-49ea-ac6a-f74d2c8064cc  multilinearregression  online  DEPLOY_SUCCESS  2019-11-15T11:57:11.830Z  scikit-learn-0.20  model
d6a9f5b8-cd45-4ab5-bfc0-16ad50012fbd  SIP-navi bayes-pipe    online  DEPLOY_SUCCESS  2019-11-14T11:51:53.862Z  scikit-learn-0.20  model
------------------------------------  ---------------------  ------  --------------  ------------------------  -----------------  -------------
